Import libraries

In [1]:
import pandas as pd

In [2]:
import data_prep as dp

Import Data

In [3]:
dat_val, dat_tr = dp.load_train_split()

Path to dataset files: /Users/anabellafalk/.cache/kagglehub/datasets/kartik2112/fraud-detection/versions/1


### Prepare Data

In [18]:
# Initialize relevant columns
model_cols = ['category', 'amt', 'trans_month', 'trans_year']
identifier_cols = ['state', 'age_at_trans', 'city_pop', 'job_bin']
other_cols = ['distance']

#### Extract Date Info

In [5]:
# Create month column
dp.extract_month(dat_tr)

In [6]:
# Create year column
dp.extract_year(dat_tr)

In [7]:
# Create month and year column
dp.extract_month_year(dat_tr)

#### Extract Age

In [8]:
dp.extract_age(dat_tr)

#### Bin Jobs

In [9]:
# Fit binning on training data
job_bin_map = dp.fit_job_bins(dat_tr)

In [10]:
# Apply bins to training data
dp.bin_jobs(dat_tr, job_bin_map)

#### Extract Distance

In [11]:
dp.extract_distance(dat_tr)

#### Scale Numeric Data

#### Encode Categorical Data

### Fit Models

#### Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
rfc = RandomForestClassifier()

In [19]:
rfc.fit(dat_tr[model_cols + identifier_cols + other_cols], dat_tr['is_fraud'])

ValueError: could not convert string to float: 'grocery_pos'

#### SVM

scale features, < 100, use kernel, balance classes

#### XGBoost

#### Logistic Regression